In [ ]:
from nntool.api import NNGraph
from nntool_generate_model import build_graph

from PIL import Image

from quantization.utils import hwc_slice
import numpy as np

In [ ]:
G = build_graph("../weights/yolox-QVGA-bayer-BGR.onnx", "../weights/precalculated_stats_bayer_v2_qvga_trainset_nonpadded.pickle")

In [ ]:
image = Image.open("../test_data/input.ppm")
input_tensor = hwc_slice(np.array(image))
print(input_tensor.shape)
flout = G.execute([input_tensor])
dqout = G.execute([input_tensor], dequantize=True)
qqout = G.execute([input_tensor], dequantize=False, quantize=True)
res = G.execute_on_target(
    input_tensors=qqout[0],
    directory="/tmp/test_yolox_people_detection",
    settings={
        "tensor_directory": "tensors",
        "model_directory": "",
        "model_file": "ATmodel.c",

        "l1_size": 128000,
        "l2_size": 1000000,

        "graph_monitor_cycles": True,
        "graph_produce_node_names": True,
        "graph_produce_operinfos": True,
        "graph_const_exec_from_flash": True,
        "graph_dump_tensor_to_file": True,

        "l3_ram_device": "AT_MEM_L3_DEFAULTRAM",
        "l3_flash_device": "AT_MEM_L3_DEFAULTFLASH", #"AT_MEM_L3_DEFAULTFLASH",
    },
    check_on_target=True,
    output_tensors=4,
    print_output=True
)

In [ ]:
from nntool.api.utils import qsnr

for i, (a, b) in enumerate(zip(res.output_tensors, qqout)):
    for idx in range(len(a)):
        if a[idx] is not None and b[idx] is not None:
            print(f"{i:5} [{idx:2}] --> {qsnr(a[idx], b[idx])}")

In [ ]:
G.draw(quant_labels=True)